```
dx download Annotation/gnomad_Freq/gnomad_v2.1_sv.sites.sorted.tsv
dx download chr-subset-genes-variants/all_variants_chr21_grch37.tsv
```

In [12]:
library(dplyr)
library(GenomicRanges)

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, 

In [1]:
gnomad = read.table('gnomad_v2.1_sv.sites.sorted.tsv', as.is=TRUE)
head(gnomad)

,V1,V2,V3,V4,V5,V6
,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,chr1,10642,10643,<BND>,gnomAD-SV_v2.1_BND_1_1,0.006786
2,chr1,21000,26000,<DEL>,gnomAD-SV_v2.1_DEL_1_1,0.016073
3,chr1,40000,47200,<DUP>,gnomAD-SV_v2.1_DUP_1_1,0.068963
4,chr1,46000,111250,"<CN=0>,<CN=1>,<CN=2>,<CN=3>,<CN=4>,<CN=5>,<CN=6>,<CN=7>,<CN=8>",gnomAD-SV_v2.1_MCNV_1_1,"0.09479,0.270108,0.349762,0.177452,0.083632,0.019986,0.003784,0.000388,9.7e-05"
5,chr1,50941,51053,<DEL>,gnomAD-SV_v2.1_DEL_1_2,0.004052
6,chr1,54665,54666,<INS>,gnomAD-SV_v2.1_INS_1_1,9.4e-05


In [7]:
colnames(gnomad) = c('chr', 'start', 'end', 'type', 'id', 'af')
head(gnomad)

,chr,start,end,type,id,af
,<chr>,<int>,<int>,<chr>,<chr>,<chr>
1,chr1,10642,10643,<BND>,gnomAD-SV_v2.1_BND_1_1,0.006786
2,chr1,21000,26000,<DEL>,gnomAD-SV_v2.1_DEL_1_1,0.016073
3,chr1,40000,47200,<DUP>,gnomAD-SV_v2.1_DUP_1_1,0.068963
4,chr1,46000,111250,"<CN=0>,<CN=1>,<CN=2>,<CN=3>,<CN=4>,<CN=5>,<CN=6>,<CN=7>,<CN=8>",gnomAD-SV_v2.1_MCNV_1_1,"0.09479,0.270108,0.349762,0.177452,0.083632,0.019986,0.003784,0.000388,9.7e-05"
5,chr1,50941,51053,<DEL>,gnomAD-SV_v2.1_DEL_1_2,0.004052
6,chr1,54665,54666,<INS>,gnomAD-SV_v2.1_INS_1_1,9.4e-05


In [8]:
table(gnomad$type)

In [11]:
gnomad = gnomad %>% mutate(svtype=ifelse(grepl('CN', type), 'CNV', type),
                           svtype=ifelse(grepl('INS', type), 'INS', svtype),
                           svtype=ifelse(grepl('DEL', type), 'DEL', svtype),
                           svtype=ifelse(grepl('DUP', type), 'DUP', svtype),
                           svtype=ifelse(grepl('INV', type), 'INV', svtype))
head(gnomad)                

,chr,start,end,type,id,af,svtype
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,chr1,10642,10643,<BND>,gnomAD-SV_v2.1_BND_1_1,0.006786,<BND>
2,chr1,21000,26000,<DEL>,gnomAD-SV_v2.1_DEL_1_1,0.016073,DEL
3,chr1,40000,47200,<DUP>,gnomAD-SV_v2.1_DUP_1_1,0.068963,DUP
4,chr1,46000,111250,"<CN=0>,<CN=1>,<CN=2>,<CN=3>,<CN=4>,<CN=5>,<CN=6>,<CN=7>,<CN=8>",gnomAD-SV_v2.1_MCNV_1_1,"0.09479,0.270108,0.349762,0.177452,0.083632,0.019986,0.003784,0.000388,9.7e-05",CNV
5,chr1,50941,51053,<DEL>,gnomAD-SV_v2.1_DEL_1_2,0.004052,DEL
6,chr1,54665,54666,<INS>,gnomAD-SV_v2.1_INS_1_1,9.4e-05,INS


In [37]:
gnomad$af.max = unlist(lapply(strsplit(gnomad$af, split=','), function(x) max(as.numeric(x))))
head(gnomad)

,chr,start,end,type,id,af,svtype,af.max
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,chr1,10642,10643,<BND>,gnomAD-SV_v2.1_BND_1_1,0.006786,<BND>,0.006786
2,chr1,21000,26000,<DEL>,gnomAD-SV_v2.1_DEL_1_1,0.016073,DEL,0.016073
3,chr1,40000,47200,<DUP>,gnomAD-SV_v2.1_DUP_1_1,0.068963,DUP,0.068963
4,chr1,46000,111250,"<CN=0>,<CN=1>,<CN=2>,<CN=3>,<CN=4>,<CN=5>,<CN=6>,<CN=7>,<CN=8>",gnomAD-SV_v2.1_MCNV_1_1,"0.09479,0.270108,0.349762,0.177452,0.083632,0.019986,0.003784,0.000388,9.7e-05",CNV,0.349762
5,chr1,50941,51053,<DEL>,gnomAD-SV_v2.1_DEL_1_2,0.004052,DEL,0.004052
6,chr1,54665,54666,<INS>,gnomAD-SV_v2.1_INS_1_1,9.4e-05,INS,0.000094


In [16]:
vars.df = read.table('all_variants_chr21_grch37.tsv', as.is=TRUE, header=TRUE)
head(vars.df)

,chr,start,end,type,variant_id
,<int>,<int>,<int>,<chr>,<chr>
1,21,1,23875167,DEL,nssv1613109
2,21,1,9411300,DEL,nssv3000025
3,21,9410851,9483551,DUP,essv25939335
4,21,9411194,9841977,DEL,nssv14267892
5,21,9411194,9841977,DEL,nssv14267893
6,21,9411194,9841977,DEL,nssv14267894


In [17]:
table(vars.df$type)


   CNV    DEL    DUP    INS    INV 
 24671 219242  48684 118236   9147 

In [41]:
vars.af = lapply(unique(vars.df$type), function(tt){
    vars.gr = vars.df %>% mutate(chr=paste0('chr', chr)) %>% filter(type==tt) %>% makeGRangesFromDataFrame(keep.extra.columns = TRUE)
    gnomad.gr = gnomad %>% filter(svtype==tt) %>% makeGRangesFromDataFrame(keep.extra.columns = TRUE)
    findOverlaps(vars.gr, gnomad.gr) %>% as.data.frame %>%
    mutate(af=gnomad.gr$af.max[subjectHits], variant_id=vars.gr$variant_id[queryHits],
          qw=width(vars.gr)[queryHits], sw=width(gnomad.gr)[subjectHits],
          qsw=width(pintersect(vars.gr[queryHits], gnomad.gr[subjectHits]))) %>%
    filter(qsw>.9 * qw, qsw > .9 * sw) %>% 
    group_by(variant_id) %>% summarize(af=max(af))
}) %>% bind_rows()

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



In [42]:
head(vars.af)

variant_id,af
<chr>,<dbl>
essv100052,0.298128
essv100397,0.298128
essv100590,0.298128
essv101052,0.298128
essv101179,0.298128
essv101524,0.298128


In [43]:
summary(vars.af$af)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000046 0.128653 0.384190 0.350648 0.493359 0.999515 

In [44]:
write.table(vars.af, file='gnomad-af-variants-chr21.tsv', quote=FALSE, row.names=FALSE, sep='\t')

```
dx upload gnomad-af-variants-chr21.tsv --path chr-subset-genes-variants/
dx upload gnomad-variants-af.ipynb --path chr-subset-genes-variants/
```